In [97]:
import os
import time
import datetime
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
temp_data_1 = pd.read_excel('./datas/First_cut_ExampleData.xlsx')
temp_data_1.head()

,Day,Pattern,Variant,ApplicableDays,TrainType,Vervoerder,TrainCharacteristic,Trainnumber,Location,Past_Location,Activity,Ordernumber,PlannedTime,Track,BareDrivingTime,SlackPercentage,TractionType,TimetableSpeed
0,2017-09-05,A1500,280,JJJJJNN,R,NS-Reizigers,IC,1522,Dv,NaN,V,1,2017-09-05 07:18:00,1,NaN,NaN,ICM7,140
1,2017-09-05,A1500,280,JJJJJNN,R,NS-Reizigers,IC,1522,Twl,Dv,D,2,2017-09-05 07:21:00,2,227.0,NaN,ICM7,140
2,2017-09-05,A1500,280,JJJJJNN,R,NS-Reizigers,IC,1522,Apdo,Twl,D,3,2017-09-05 07:25:00,2,194.0,NaN,ICM7,140
3,2017-09-05,A1500,280,JJJJJNN,R,NS-Reizigers,IC,1522,Apda,Apdo,D,4,2017-09-05 07:26:00,DD,67.0,NaN,ICM7,140
4,2017-09-05,A1500,280,JJJJJNN,R,NS-Reizigers,IC,1522,Apd,Apda,A,5,2017-09-05 07:28:00,1,73.0,"7,0",ICM7,140


In [3]:
temp_data_2 = pd.read_excel('./datas/Data_RAS/infra_overview.xlsx')
temp_data_2.head()

,FromLocation,FullNameFromLocation,ToLocation,FullNameToLocation,Hectometers,AverageAllowedSpeed,NumberOfParallelTracks,Diesel/Electric
0,Ac,Abcoude,Ashd,Amsterdam Holendrecht,23,120,4,E
1,Ac,Abcoude,Bkl,Breukelen,125,120,4,E
2,Adh,Achter de Hoven,Lw,Leeuwarden,20,100,1,D
3,Adh,Achter de Hoven,Lwc,Leeuwarden Camminghaburen,17,100,1,D
4,Ah,Arnhem,Ahp,Arnhem Velperpoort,13,100,2,E


## 距离表

In [4]:
locations_dis = pd.read_csv('./datas/Data_RAS/LocationDistances.csv')
locations_dis.head()

,from,to,distance
0,Ods,Dgrw,0.3
1,Dgrw,Asdma,1.9
2,Asdma,Asdm,0.9
3,Asdm,Asa,2.1
4,Asa,Dvd,2.9


In [5]:
locations = locations_dis.groupby(by=['from'])
len(locations)

472

## 时间表(可能用不到)

In [6]:
time_tables = pd.read_csv('./datas/Data_RAS/TimeTable.csv')
time_tables.head()

,Day,Trainnumber,Direction,Location,Abroad,Activity,Ordernumber,PlannedTime,TrainCharacteristic,Pattern,BareDrivingTime,TractionType,TimetableSpeed
0,4-9-2017,511,O,Zl,N,V,1.0,4-9-2017 05:44,IC,B500,NaN,ICM12,140.0
1,4-9-2017,511,O,Zlgea,N,D,2.0,4-9-2017 05:45,IC,B500,124.0,ICM12,140.0
2,4-9-2017,511,O,Hea,N,D,3.0,4-9-2017 05:47,IC,B500,123.0,ICM12,140.0
3,4-9-2017,511,O,Ddv,N,D,4.0,4-9-2017 05:52,IC,B500,287.0,ICM12,140.0
4,4-9-2017,511,O,Mp,N,K_A,5.0,4-9-2017 06:00,IC,B500,399.0,ICM12,140.0


## 测试数据集

In [40]:
datas = pd.read_csv('./datas/Data_RAS/ExampleTestSet_0800.csv')
datas.head()

,TrafficDate,Trainseries,TrainCharacteristic,Trainnumber,Location,Activity,PlannedTime,Realisation,Delay_min,DelayJump
0,2017-11-07,500E,IC,512,Ut,V,2017-11-07 06:18:00,2017-11-07 06:18:16,0,0
1,2017-11-07,500E,IC,512,Utwa,D,2017-11-07 06:19:00,2017-11-07 06:20:26,1,1
2,2017-11-07,500E,IC,512,Utlr,D,2017-11-07 06:20:00,2017-11-07 06:21:55,1,0
3,2017-11-07,500E,IC,512,Utt,D,2017-11-07 06:21:00,2017-11-07 06:22:54,1,0
4,2017-11-07,500E,IC,512,Vtn,D,2017-11-07 06:22:00,2017-11-07 06:23:53,1,0


## 1. 数据预处理

### 1.1 读取数据

`The activity of this train. Where a V stands for a departure, a D stands for a passage, and an A stands for an arrival. A K can be added in front to indicate a short stop, which is an arrival and departure planned within the same minute.`

In [8]:
final_datas = pd.read_csv('./datas/Data_RAS/RealisationData.csv')
final_datas = final_datas.drop(columns=['Cause'])

In [41]:
# 查看一下K_A和K_V的时间变换，发现较难处理，因此做简单处理
activities = datas.groupby(by=['Activity'])
for group, _ in activities:
    print(group)
#     if group == 'K_A' or group == 'K_V':
#         print(_)

A
D
K_A
K_V
V


### 1.2 取合理的延时时间范围

In [42]:
process_data = datas[(datas['Delay_min'] > -20) & (datas['Delay_min'] < 30)]
process_data

,TrafficDate,Trainseries,TrainCharacteristic,Trainnumber,Location,Activity,PlannedTime,Realisation,Delay_min,DelayJump
0,2017-11-07,500E,IC,512,Ut,V,2017-11-07 06:18:00,2017-11-07 06:18:16,0,0
1,2017-11-07,500E,IC,512,Utwa,D,2017-11-07 06:19:00,2017-11-07 06:20:26,1,1
2,2017-11-07,500E,IC,512,Utlr,D,2017-11-07 06:20:00,2017-11-07 06:21:55,1,0
3,2017-11-07,500E,IC,512,Utt,D,2017-11-07 06:21:00,2017-11-07 06:22:54,1,0
4,2017-11-07,500E,IC,512,Vtn,D,2017-11-07 06:22:00,2017-11-07 06:23:53,1,0
...,...,...,...,...,...,...,...,...,...,...
14119,2017-11-07,810000O,LM,810515,Ns,D,2017-11-07 06:24:00,2017-11-07 06:23:42,0,0
14120,2017-11-07,810000O,LM,810515,Hde,D,2017-11-07 06:29:00,2017-11-07 06:27:54,-1,-1
14121,2017-11-07,810000O,LM,810515,Htba,D,2017-11-07 06:35:00,2017-11-07 06:34:11,0,1
14122,2017-11-07,810000O,LM,810515,Zlra,D,2017-11-07 06:36:00,2017-11-07 06:35:16,0,0


In [43]:
activities = process_data.groupby(by=['Delay_min'])
# for group, _ in activities:
#     print(group)

### 1.3 对时间进行转换为时间戳 (分两次使用，进行结果对比)

In [44]:
def todatetime(data):
    if isinstance(data, str):
        pass
    else:
        data = str(data)
    return int(time.mktime(time.strptime(data, '%Y-%m-%d %H:%M:%S')))

In [45]:
process_data['PlannedTime'] = process_data['PlannedTime'].apply(todatetime)

In [46]:
process_data['Realisation'] = process_data['Realisation'].apply(todatetime)
process_data

,TrafficDate,Trainseries,TrainCharacteristic,Trainnumber,Location,Activity,PlannedTime,Realisation,Delay_min,DelayJump
0,2017-11-07,500E,IC,512,Ut,V,1510006680,1510006696,0,0
1,2017-11-07,500E,IC,512,Utwa,D,1510006740,1510006826,1,1
2,2017-11-07,500E,IC,512,Utlr,D,1510006800,1510006915,1,0
3,2017-11-07,500E,IC,512,Utt,D,1510006860,1510006974,1,0
4,2017-11-07,500E,IC,512,Vtn,D,1510006920,1510007033,1,0
...,...,...,...,...,...,...,...,...,...,...
14119,2017-11-07,810000O,LM,810515,Ns,D,1510007040,1510007022,0,0
14120,2017-11-07,810000O,LM,810515,Hde,D,1510007340,1510007274,-1,-1
14121,2017-11-07,810000O,LM,810515,Htba,D,1510007700,1510007651,0,1
14122,2017-11-07,810000O,LM,810515,Zlra,D,1510007760,1510007716,0,0


### 1.4 加入距离参数

In [47]:
# 得到一个对应关系，方便匹配，采用哈希存储，降低时间复杂度
locations_datas = locations_dis.values.tolist()
locations_json = {}
for locations_data in locations_datas:
    if locations_data[0] not in locations_json.keys():
        locations_json[locations_data[0]] = [[locations_data[1], locations_data[2]]]
    else:
        locations_json[locations_data[0]].append([locations_data[1], locations_data[2]])
# locations_json

In [48]:
extract_numbers = process_data[['Trainnumber', 'Location']].values.tolist()

In [49]:
distances = []
got_number = []
departure = ''

for index, extract_number in enumerate(extract_numbers):
    if extract_number[0] not in got_number:
        got_number.append(extract_number[0])
        distance = 0
    else:
        distance = -1.
        try:
            dises = locations_json[extract_numbers[index-1][1]]
            for dis in dises:
                if dis[0] == extract_number[1]:
                    distance = dis[1]
                    break
        except:
            pass
    distances.append(distance)
process_data['distance'] = distances
process_data

,TrafficDate,Trainseries,TrainCharacteristic,Trainnumber,Location,Activity,PlannedTime,Realisation,Delay_min,DelayJump,distance
0,2017-11-07,500E,IC,512,Ut,V,1510006680,1510006696,0,0,0.0
1,2017-11-07,500E,IC,512,Utwa,D,1510006740,1510006826,1,1,1.4
2,2017-11-07,500E,IC,512,Utlr,D,1510006800,1510006915,1,0,1.9
3,2017-11-07,500E,IC,512,Utt,D,1510006860,1510006974,1,0,1.7
4,2017-11-07,500E,IC,512,Vtn,D,1510006920,1510007033,1,0,2.1
...,...,...,...,...,...,...,...,...,...,...,...
14119,2017-11-07,810000O,LM,810515,Ns,D,1510007040,1510007022,0,0,12.1
14120,2017-11-07,810000O,LM,810515,Hde,D,1510007340,1510007274,-1,-1,8.6
14121,2017-11-07,810000O,LM,810515,Htba,D,1510007700,1510007651,0,1,-1.0
14122,2017-11-07,810000O,LM,810515,Zlra,D,1510007760,1510007716,0,0,1.7


In [50]:
# 将没有距离的数据去掉
process_data = process_data[process_data['distance'] != -1]
process_data

,TrafficDate,Trainseries,TrainCharacteristic,Trainnumber,Location,Activity,PlannedTime,Realisation,Delay_min,DelayJump,distance
0,2017-11-07,500E,IC,512,Ut,V,1510006680,1510006696,0,0,0.0
1,2017-11-07,500E,IC,512,Utwa,D,1510006740,1510006826,1,1,1.4
2,2017-11-07,500E,IC,512,Utlr,D,1510006800,1510006915,1,0,1.9
3,2017-11-07,500E,IC,512,Utt,D,1510006860,1510006974,1,0,1.7
4,2017-11-07,500E,IC,512,Vtn,D,1510006920,1510007033,1,0,2.1
...,...,...,...,...,...,...,...,...,...,...,...
14117,2017-11-07,810000O,LM,810515,Nkk,D,1510006080,1510006121,0,0,5.3
14119,2017-11-07,810000O,LM,810515,Ns,D,1510007040,1510007022,0,0,12.1
14120,2017-11-07,810000O,LM,810515,Hde,D,1510007340,1510007274,-1,-1,8.6
14122,2017-11-07,810000O,LM,810515,Zlra,D,1510007760,1510007716,0,0,1.7


### 1.5 得出上一站的延迟时间

In [53]:
extract_minines = process_data[['Trainnumber', 'Delay_min']].values.tolist()
extract_minines

[[512, 0],
 [512, 1],
 [512, 1],
 [512, 1],
 [512, 1],
 [512, 1],
 [512, 0],
 [512, 1],
 [512, 1],
 [512, 1],
 [512, 1],
 [512, 1],
 [512, 1],
 [512, 1],
 [512, 2],
 [512, 1],
 [512, 2],
 [512, 1],
 [512, 2],
 [512, 2],
 [512, 3],
 [512, 3],
 [512, 3],
 [512, 2],
 [512, 3],
 [512, 2],
 [512, 3],
 [512, 3],
 [512, 3],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 1],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 1],
 [516, 1],
 [516, 0],
 [516, 1],
 [516, 1],
 [516, 0],
 [516, 1],
 [516, 1],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 1],
 [516, 1],
 [516, 1],
 [516, 1],
 [516, 0],
 [516, 0],
 [516, 0],
 [516, 1],
 [516, 1],
 [520, 0],
 [520, 1],
 [520, 1],
 [520, 0],
 [520, 0],
 [520, 0],
 [520, 0],
 [520, 0],
 [520, 1],
 [520, 1],

In [56]:
pre_delays = []
got_number = []
departure = ''

temp_number = extract_minines[0][0]
print(temp_number)
for index, extract_minine in enumerate(extract_minines):
    if temp_number != extract_minine[0]:
#         got_number.append(extract_number[0])
        temp_number = extract_minine[0]
        pre_delay = 0
    else:
        pre_delay = -1.
        try:
            pre_delay = extract_minines[index-1][1]
        except:
            pass
    pre_delays.append(pre_delay)
process_data['PreDelay_min'] = pre_delays
process_data['PreDelay_min'][0] = 0
process_data

512


,TrafficDate,Trainseries,TrainCharacteristic,Trainnumber,Location,Activity,PlannedTime,Realisation,Delay_min,DelayJump,distance,PreDelay_min
0,2017-11-07,500E,IC,512,Ut,V,1510006680,1510006696,0,0,0.0,0
1,2017-11-07,500E,IC,512,Utwa,D,1510006740,1510006826,1,1,1.4,0
2,2017-11-07,500E,IC,512,Utlr,D,1510006800,1510006915,1,0,1.9,1
3,2017-11-07,500E,IC,512,Utt,D,1510006860,1510006974,1,0,1.7,1
4,2017-11-07,500E,IC,512,Vtn,D,1510006920,1510007033,1,0,2.1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14117,2017-11-07,810000O,LM,810515,Nkk,D,1510006080,1510006121,0,0,5.3,0
14119,2017-11-07,810000O,LM,810515,Ns,D,1510007040,1510007022,0,0,12.1,0
14120,2017-11-07,810000O,LM,810515,Hde,D,1510007340,1510007274,-1,-1,8.6,0
14122,2017-11-07,810000O,LM,810515,Zlra,D,1510007760,1510007716,0,0,1.7,-1


## 2. 制作标签建立训练集和测试集

- [pass!] 将延时时间小于等于0的置为1，其余置为0，归为二分类问题
- 做回归预测

In [98]:
# def get_label(data):
#     if data <= 0:
#         return 1
#     else:
#         return 0
    
# process_data['labels'] = process_data['Delay_min'].apply(get_label)
# process_data

In [128]:
def one_hot(data):
    if data == 'V':
        return 1
    elif data == 'A':
        return 2
    elif data == 'D':
        return 3
    elif data == 'K_A':
        return 4
    else:
        return 5

features = process_data[['Activity', 'distance', 'PreDelay_min']]
features['Activity'] = features['Activity'].apply(one_hot)
labels = process_data[['Delay_min']]

In [129]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12759 entries, 0 to 14123
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Activity      12759 non-null  int64  
 1   distance      12759 non-null  float64
 2   PreDelay_min  12759 non-null  int64  
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


In [130]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12759 entries, 0 to 14123
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Delay_min  12759 non-null  int64
dtypes: int64(1)
memory usage: 839.4 KB


### 2.2 分训练集和验证集



In [131]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=20)

### 2.3 标准化特征

对数据集进行标准化，不然MSE较大

In [132]:
ss_X = StandardScaler()
ss_y = StandardScaler()

X_train = ss_X.fit_transform(X_train)
X_test = ss_X.transform(X_test)

y_train = ss_y.fit_transform(y_train)
y_test = ss_y.transform(y_test)

## 3. 预测

In [133]:
def mse(x, y):
    return np.linalg.norm(x - y)

### 3.1 线性回归做Baseline

In [134]:
clf_linear = LinearRegression()
clf_linear.fit(X_train, y_train)
y_predict_linear = clf_linear.predict(X_test)

y_predict_linear = y_predict_linear.reshape(-1, 1)

In [135]:
y_predict_linear

array([[ 5.38993351],
       [ 0.24940764],
       [ 0.25014067],
       ...,
       [-0.10194206],
       [-0.09104615],
       [-0.43013331]])

In [136]:
mse_error_linear = mse(y_predict_linear, y_test)
print(f'The MSE Error is :{mse_error:.4f}')

The MSE Error is :23.6464


In [137]:
# 使用R-squared、MSE和MAE指标对三种配置的支持向量机（回归）模型在相同测试集上进行性能评估。
print(f'R-squared value of linear regression is {clf.score(X_test, y_test):.4f}')
print(f'The mean squared error of linear regression is {mean_squared_error(ss_y.inverse_transform(y_test), ss_y.inverse_transform(y_predict)):.4f}')
print(f'The mean absoluate error of linear regression is {mean_absolute_error(ss_y.inverse_transform(y_test), ss_y.inverse_transform(y_predict)):.4f}')

R-squared value of linear regression is 0.8616
The mean squared error of linear regression is 1.0768
The mean absoluate error of linear regression is 0.5656


### 3.2 SVM预测

- 采用线性基

In [138]:
clf = SVR(C=1.2, kernel = "linear", gamma=0.2)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)

y_predict = y_predict.reshape(-1, 1)

mse_error = mse(y_predict, y_test)
print(f'The MSE Error is :{mse_error}')

The MSE Error is :23.64641460046781


In [139]:
print(f'R-squared value of linear SVR is {clf.score(X_test, y_test):.4f}')
print(f'The mean squared error of linear SVR is {mean_squared_error(ss_y.inverse_transform(y_test), ss_y.inverse_transform(y_predict)):.4f}')
print(f'The mean absoluate error of linear SVR is {mean_absolute_error(ss_y.inverse_transform(y_test), ss_y.inverse_transform(y_predict)):.4f}')

R-squared value of linear SVR is 0.8616
The mean squared error of linear SVR is 1.0768
The mean absoluate error of linear SVR is 0.5656


- 采用径向基核函数

……

## 4. 加入时间变量做预测

### 4.1 分离特征和标签，特征中加入时间变量

In [141]:
features = process_data[['Activity', 'distance', 'PreDelay_min', 'PlannedTime', 'Realisation']]
features['Activity'] = features['Activity'].apply(one_hot)
labels = process_data[['Delay_min']]

### 4.2 分数据集

In [142]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=20)

### 4.3 标准化

In [143]:
ss_X = StandardScaler()
ss_y = StandardScaler()

X_train = ss_X.fit_transform(X_train)
X_test = ss_X.transform(X_test)

y_train = ss_y.fit_transform(y_train)
y_test = ss_y.transform(y_test)

In [144]:
clf = SVR(C=1.2, kernel = "linear", gamma=0.2)
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)

y_predict = y_predict.reshape(-1, 1)

mse_error = mse(y_predict, y_test)
print(f'The MSE Error is :{mse_error}')

print(f'R-squared value of linear SVR is {clf.score(X_test, y_test):.4f}')
print(f'The mean squared error of linear SVR is {mean_squared_error(ss_y.inverse_transform(y_test), ss_y.inverse_transform(y_predict)):.4f}')
print(f'The mean absoluate error of linear SVR is {mean_absolute_error(ss_y.inverse_transform(y_test), ss_y.inverse_transform(y_predict)):.4f}')

The MSE Error is :8.424247399540059
R-squared value of linear SVR is 0.9824
The mean squared error of linear SVR is 0.1367
The mean absoluate error of linear SVR is 0.2849
